In [524]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier # 60
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier

In [525]:
!pip install lightgbm

You should consider upgrading via the 'c:\users\sm2142\anaconda3\python.exe -m pip install --upgrade pip' command.


In [526]:
!pip install xgboost

You should consider upgrading via the 'c:\users\sm2142\anaconda3\python.exe -m pip install --upgrade pip' command.


# 문제 정의

In [527]:
# 문제 정의
# 설문조사를 통한 강아지 추천


In [528]:
 pd.DataFrame({'이름':[v1],'나이':['']})

,이름,나이
0,S,


# 데이터 수집하기

In [529]:

dog = pd.read_csv('TotalDog.csv', encoding='euc-kr')
dog['이름'].value_counts()

# pd.set_option('display.max_row', 2000)
# pd.set_option('display.max_columns', 2000)


슈나우저       100
말티즈         63
푸들          58
믹스견         57
진돗개         57
포메라니안       56
시츄          54
허스키         53
치와와         53
미니핀         52
코카스파니엘      52
러프콜리        51
요크셔테리어      51
달마시안        51
비숑          51
웰시코기        51
차우차우        50
불독          50
그레이 하운드     50
사모예드        50
아키타         50
시바견         50
도베르만        50
닥스훈트        50
골든 리트리버     50
보더콜리        50
말라뮤트        50
미니어쳐 핀셔     50
파피용         50
삽살개         50
풍산개         50
셰퍼트         50
포메리안         1
말티지          1
요키           1
발바리          1
Name: 이름, dtype: int64

In [530]:
dog

,이름,크기,거주환경,실내실외,외로움,운동량,털빠짐,친화성(애교),의료비용,개짖음,집지키기
0,그레이 하운드,L,A,I,4,4,2,5,10u,4,1
1,그레이 하운드,L,A,I,4,4,3,5,30o,4,4
2,그레이 하운드,L,A,I,3,4,3,3,30o,2,1
3,그레이 하운드,L,A,I,4,4,2,5,10u,1,3
4,그레이 하운드,L,A,I,4,4,2,4,30u,3,2
...,...,...,...,...,...,...,...,...,...,...,...
1709,골든 리트리버,L,J,IO,2,4,5,4,10u,3,2
1710,골든 리트리버,M,J,IO,1,3,5,5,10u,2,2
1711,골든 리트리버,L,J,IO,2,3,4,5,30o,3,3
1712,골든 리트리버,M,J,IO,1,4,5,5,10u,2,3


# 데이터 전처리

In [531]:

dog['의료비용'] = dog['의료비용'].replace('35u','30u')
dog['의료비용'] = dog['의료비용'].replace('40u','30o')
dog['의료비용'] = dog['의료비용'].replace('35o','30o')
dog['이름'] = dog['이름'].replace('말티지','말티즈')
dog['이름'] = dog['이름'].replace('발바리','믹스견')
dog['이름'] = dog['이름'].replace('요키','믹스견')
dog['이름'] = dog['이름'].replace('포메리안','포메라니안')
dog['이름'].unique()
dog['이름'].value_counts()


dog['이름'] = dog[dog["이름"] !="슈나우저"]
dog['이름'] = dog[dog["이름"] !="비숑"]
dog['이름'] = dog[dog["이름"] !="시바견"]
dog['이름'] = dog[dog["이름"] !="삽살개"]
dog['이름'] = dog[dog["이름"] !="사모예드"]
dog['이름'] = dog[dog["이름"] !="아키타"]
dog['이름'] = dog[dog["이름"] !="그레이 하운드"]
dog['이름'] = dog[dog["이름"] !="도베르만"]
dog.dropna(inplace=True)

Title_Dictionary = {
     "말티즈" : 0,
    "믹스견":1,
    "푸들" : 2,
    "포메라니안" : 3,
    "진돗개": 4,
    "시츄":5,
    "허스키" : 6,
    "치와와":7,
    "미니핀": 8,
    "코카스파니엘": 9,
#    "비숑":10,
    "요크셔테리어" : 11,
     "웰시코기":12,
    "러프콜리": 13,
   "달마시안": 14,
    "골든 리트리버" : 15,
#     "도베르만" : 16,
#     "사모예드" : 17
#      "그레이 하운드" : 18,
    "미니어쳐 핀셔" : 19,
#    "시바견" : 20,
      "파피용" : 21,
#      "삽살개" : 22,
     "셰퍼트" :  23,
#     "아키타" : 24
     "차우차우" : 25,
     "닥스훈트" : 26,
   "보더콜리" : 27,
     "말라뮤트" : 28,
    "불독" : 29,
   "풍산개" : 30
#      "슈나우저" : 31,
    
}

dog['이름'] = dog['이름'].map(Title_Dictionary)

onehot_column = ['크기','거주환경','실내실외','의료비용']


for col in onehot_column:
    dummy = pd.get_dummies(dog[col],prefix = col)
    # 원핫인코딩된 결과물을 기존의 자료와 합치기
    dog = pd.concat([dog,dummy], axis = 1)
    # 글자형 컬럼(원핫인코딩 진행한) 삭제
    dog.drop(col,axis = 1, inplace = True)

dummy

,의료비용_10u,의료비용_30o,의료비용_30u
50,1,0,0
51,1,0,0
52,1,0,0
53,0,0,1
54,1,0,0
...,...,...,...
1709,1,0,0
1710,1,0,0
1711,0,1,0
1712,1,0,0


In [532]:
dog['이름'].value_counts()

0     64
1     59
2     58
3     57
4     57
5     54
6     53
7     53
8     52
9     52
11    51
12    51
13    51
14    51
29    50
15    50
19    50
21    50
23    50
25    50
26    50
27    50
28    50
30    50
Name: 이름, dtype: int64

In [533]:
dog.head()

,이름,외로움,운동량,털빠짐,친화성(애교),개짖음,집지키기,크기_L,크기_M,크기_S,거주환경_A,거주환경_J,실내실외_I,실내실외_IO,실내실외_O,의료비용_10u,의료비용_30o,의료비용_30u
50,26,5,4,3,3,4,3,0,1,0,1,0,1,0,0,1,0,0
51,26,4,5,5,3,3,3,0,1,0,1,0,1,0,0,1,0,0
52,26,3,4,5,3,4,3,0,0,1,1,0,1,0,0,1,0,0
53,26,4,3,4,3,5,4,0,1,0,1,0,1,0,0,0,0,1
54,26,4,4,5,3,3,2,0,1,0,1,0,1,0,0,1,0,0


# 모델 선택 및 하이퍼파라미터 튜닝
# 문제와 정답으로 분리 - X, y

In [534]:
y = dog.iloc[:,0]
X = dog.iloc[:,1:]
ans = dog.iloc[5,1:]


X_train, X_test, y_train, y_test = train_test_split(
                                X, #문제데이터
                                y,
                                test_size = 0.3, 
                                random_state = 3 
)


# 학습, 평가

In [535]:
test_list= []
train_list= []  # n_neighbors = 2
for i in range(1,101):
    
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train,y_train)
    
    train_list.append(model.score(X_train,y_train))

train_list

for i in range(1,101):
    
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train,y_train)
    
    test_list.append(model.score(X_test,y_test))

test_list

knn_model = KNeighborsClassifier(n_neighbors= 12,
                                 p=1,
                                 weights = 'distance'
                                )
knn_model

knn_model.fit(X_train,y_train)
knn_pred = knn_model.predict(X_test)
# print('XGB : %.2f'%(metrics.accuracy_score(knn_pred,y_test)*100))


# RandomForestClassifier


scaler = MinMaxScaler()
scaler.fit(X_train)
trans_X_train = scaler.transform(X_train)
trans_X_test = scaler.transform(X_test)
Rand = RandomForestClassifier()
Rand.fit(trans_X_train,y_train)
Rand_pred = Rand.predict(X_test)
# print('Random : %.2f'%(metrics.accuracy_score(Rand_pred,y_test)*100))

# LGBMClassifier


LGB = LGBMClassifier()
LGB.fit(X_train,y_train)
LGB_pred = LGB.predict(X_test)
# LGB.score(X_test,y_test)
# print('LGB : %.2f'%(metrics.accuracy_score(LGB_pred,y_test)*100))

# XGBClassifier

XGB = XGBClassifier(max_depth= 15,
                    max_features= 0.5, 
                    max_leaf_nodes= 20,
                    min_samples_leaf= 10, 
                    n_estimators= 1000)
XGB.fit(X_train,y_train)
XGB_pred = XGB.predict(X_test)
# print('XGB : %.2f'%(metrics.accuracy_score(XGB_pred,y_test)*100))


# tree
tree = DecisionTreeClassifier()
result = cross_val_score(tree, X_train, y_train, cv = 5)
result.mean()
tree = DecisionTreeClassifier()
tree.fit(X_train,y_train)
tree.score(X_train,y_train)
tree.score(X_test,y_test)


# LinearSVC

svm = LinearSVC()
svm.fit(X_train,y_train)
svm_pred = svm.predict(X_test)
# print('SVM: %.2f'%(metrics.accuracy_score(svm_pred,y_test)*100))


 # DecisionTreeClassifier
dog_dtc = DecisionTreeClassifier()
dog_dtc.fit(X_train,y_train)
Desi_pred = dog_dtc.predict(X_test)
# print('Decision: %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))

# # KNeighborsClassifier
# dog_knn = KNeighborsClassifier(n_neighbors=3)
# dog_knn.fit(X_train,y_train)
# dog_pred = dog_knn.predict(X_test)
# # print('Knn: %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))

# LogisticRegression
dog_lrg = LogisticRegression()
dog_lrg.fit(X_train,y_train)
dog_pred = dog_lrg.predict(X_test)
#print(dog_pred)
print('Logistic : %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))

# dog_rfc = RandomForestClassifier()
# dog_rfc.fit(X_train,y_train)
# dog_pred = dog_rfc.predict(X_test)
# # print('RandomForest : %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))

forest = RandomForestClassifier(n_estimators=1000,
                                # tree의 개수
                                      max_features=0.7,# 각각의 트리에서 사용할 특성의 개수
                                      max_depth=5,
                                      min_samples_leaf=15,
                                      max_leaf_nodes=50)

forest.fit(X_train,y_train)
forest_pred = forest.predict(X_test)
# forest.score(X_test,y_test)


# 개발모델
knn_model = KNeighborsClassifier(n_neighbors= 12,
                                 p=1,
                                 weights = 'distance'
                                )
# XGB = XGBClassifier()
LGB = LGBMClassifier()
Rand = RandomForestClassifier(max_depth=3)

#최종모델
# Rand = RandomForestClassifier()
XGB = XGBClassifier()

knn_model.fit(X_train,y_train)
# XGB.fit(X_train,y_train)
LGB.fit(X_train,y_train)
Rand.fit(X_train,y_train)

knn_pred = knn_model.predict(X_test)
# xgb_pred = XGB.predict(X_test)
lgb_pred = LGB.predict(X_test)
rand_pred = Rand.predict(X_test)
# lgb_pred

new_data = np.array([knn_pred,rand_pred,lgb_pred])
# new_data.shape

new_data = np.transpose(new_data)
# new_data.shape

# new_data[:10]

# 스태킹 최종 결과

# Rand.fit(new_data,y_test)
# Rand_pred = Rand.predict(new_data)
XGB.fit(new_data,y_test)
xgb_pred = XGB.predict(new_data)

# print("Rand : {0:.4f}".format(accuracy_score(Rand,Rand_pred)))


print('Random : %.2f'%(metrics.accuracy_score(Rand_pred,y_test)*100))
print('knn : %.2f'%(metrics.accuracy_score(knn_pred,y_test)*100))
print('LGB : %.2f'%(metrics.accuracy_score(LGB_pred,y_test)*100))
print('XGB : %.2f'%(metrics.accuracy_score(XGB_pred,y_test)*100))
print('SVM: %.2f'%(metrics.accuracy_score(svm_pred,y_test)*100))
print('Decision: %.2f'%(metrics.accuracy_score(Desi_pred,y_test)*100))
print('forest: %.2f'%(metrics.accuracy_score(forest_pred,y_test)*100))
print('Logistic : %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))
print('RandomForest : %.2f'%(metrics.accuracy_score(dog_pred,y_test)*100))
print('스태킹 XGB : %.2f'%(metrics.accuracy_score(xgb_pred,y_test)*100))



C:\Users\SM2142\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:29:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "max_features", "max_leaf_nodes", "min_samples_leaf" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:29:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\SM2142\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\SM2142\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic : 62.01


C:\Users\SM2142\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:29:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Random : 14.78
knn : 62.53
LGB : 65.44
XGB : 65.17
SVM: 60.16
Decision: 54.88
forest: 51.72
Logistic : 62.01
RandomForest : 62.01
스태킹 XGB : 86.54


In [536]:
pred1 = xgb_pred

score1 = metrics.accuracy_score(pred1, y_test)

pred1[0]

4

In [537]:
dog1 = dog.copy()
dog1["이름"] = dog1["이름"][dog1["이름"] != pred1[0]]
dog1.dropna(inplace=True)

dog1["이름"].unique()

array([26., 14., 13., 28.,  0., 19.,  8.,  1., 29.,  5., 11., 12.,  7.,
        9., 21.,  3.,  2., 30.,  6., 15., 27., 25., 23.])

In [538]:
X = dog1.iloc[:,1:]
y = dog1.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                test_size = 0.3, 
                                random_state = 3 
)

knn_model.fit(X_train,y_train)
LGB.fit(X_train,y_train)
Rand.fit(X_train,y_train)

knn_pred = knn_model.predict(X_test)
lgb_pred = LGB.predict(X_test)
rand_pred = Rand.predict(X_test)

new_data = np.array([knn_pred,rand_pred,lgb_pred])

new_data = np.transpose(new_data)

XGB.fit(new_data, y_test)

pred2 = XGB.predict(new_data)

score2 = metrics.accuracy_score(pred2, y_test)

pred2[0]

C:\Users\SM2142\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:29:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


7.0

In [539]:
# 3째 순위는 1,2번째 순위 데이터를 제거하고 다시 학습해서 예측

In [540]:
dog2 = dog1.copy()
dog2["이름"] = dog1["이름"][dog1["이름"] != pred2[0]]
dog2.dropna(inplace=True)

dog2["이름"].unique()

array([26., 14., 13., 28.,  0., 19.,  8.,  1., 29.,  5., 11., 12.,  9.,
       21.,  3.,  2., 30.,  6., 15., 27., 25., 23.])

In [541]:
X = dog2.iloc[:,1:]
y = dog2.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                test_size = 0.3, 
                                random_state = 3 
)

knn_model.fit(X_train,y_train)
LGB.fit(X_train,y_train)
Rand.fit(X_train,y_train)

knn_pred = knn_model.predict(X_test)
lgb_pred = LGB.predict(X_test)
rand_pred = Rand.predict(X_test)

new_data = np.array([knn_pred,rand_pred,lgb_pred])

new_data = np.transpose(new_data)

XGB.fit(new_data, y_test)
pred3 = XGB.predict(new_data)

score3 = metrics.accuracy_score(pred3, y_test)

pred3[0]

C:\Users\SM2142\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:29:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


3.0

In [542]:
pred1[0], pred2[0], pred3[0]

(4, 7.0, 3.0)

In [543]:
score1,score2,score3

(0.8654353562005277, 0.850828729281768, 0.8439306358381503)

In [544]:
score1, (1-score1) * score2, (1-score1) * (1-score2) * score3

(0.8654353562005277, 0.11449146489015877, 0.016940370640229924)

In [545]:
# 임시 테스트

In [546]:
# ans

In [547]:
# ans
# ans2d = pd.DataFrame(ans)
# ans2d.iloc[:,0]
# ans = np.array(ans)
# ans = ans.reshape(1,-1)
# ans
# knn_ans = knn_model.predict(ans)
# rand_ans = Rand.predict(ans)
# lgb_ans = LGB.predict(ans)
# new_ans = np.array([knn_ans,rand_ans,lgb_ans])
# new_ans.shape
# new_ans = np.transpose(new_ans)
# new_ans.shape

# ans_end = XGB.predict(new_ans)
# ans_end


In [548]:
 pd.DataFrame({'이름':[1],'주거환경':[2],'실내실외':[3], '의료비용':[8]})

,이름,주거환경,실내실외,의료비용
0,1,2,3,8


In [579]:
# 정답지만들기
dogresult = dog.iloc[0,:]
#정답지 백지화
for i in range(len(dogresult)):
    dogresult[i] = 0
#정답 입력
if v1 == 'S':
    dogresult[9] =1
elif v1 == 'M':
    dogresult[8] =1
    

elif v1 == 'L':
    dogresult[7]= 1
if v2 == 'A':
    dogresult[10] = 1
elif v1 == 'J':
    dogresult[11] = 1
if v3 == 'I':
    dogresult[12] = 1
elif v3 == 'O':
    dogresult[14] = 1
elif v3 == 'IO':
    dogresult[13] = 1
    
dogresult[2]=v5
dogresult[3]=v6
dogresult[4]=v7

if v8 == '10u':
    dogresult[15] = 1
elif v8 == '30o':
    dogresult[16] = 1
elif v8 == '30u':
    dogresult[17] = 1
    
dogresult[5] = v10
dogresult[6] = v11
#정답컬럼 삭제
dogresult.drop('이름', inplace= True)

# dogresultlist = X
dogresult1 = np.array(dogresult)
dogresult1 = dogresult1.reshape(1,-1)


knn_dogresult = knn_model.predict(dogresult1)
rand_dogresult = Rand.predict(dogresult1)
lgb_dogresult = LGB.predict(dogresult1)
new_dogresult = np.array([knn_dogresult,rand_dogresult,lgb_dogresult])
print(new_dogresult.shape)
new_dogresult = np.transpose(new_dogresult)
print(new_dogresult.shape)

pred_1= XGB.predict(new_dogresult)
pred_1

# score2 = metrics.accuracy_score(pred2, y_test)
socore_1 = metrics.accuracy_score(pred_1,y_test)*100
# print(pred_1)
# print(socore_1)

(3, 1)
(1, 3)


<ipython-input-579-3d24387cc53e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dogresult[i] = 0
<ipython-input-579-3d24387cc53e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dogresult[9] =1
<ipython-input-579-3d24387cc53e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dogresult[10] = 1
<ipython-input-579-3d24387cc53e>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

ValueError: Found input variables with inconsistent numbers of samples: [1, 346]

In [ ]:
# 플라스크 연동

In [ ]:
from flask import Flask
from flask import request, redirect

app = Flask(__name__) 

@app.route('/dogmc', methods=['GET'])
def index():
    
    v1 = request.args['v1']
    v2 = request.args['v2']
    v3 = request.args['v3']
    v4 = request.args['v4']
    v5 = request.args['v5']
    v6 = request.args['v6']
    v7 = request.args['v7']
    v8 = request.args['v8']
    v10 = request.args['v10']
    v11 = request.args['v11']


# 정답지만들기
dogresult = dog.iloc[0,:]
#정답지 백지화
for i in range(len(dogresult)):
    dogresult[i] = 0
#정답 입력
if v1 == 'S':
    dogresult[9] =1
elif v1 == 'M':
    dogresult[8] =1
    

elif v1 == 'L':
    dogresult[7]= 1
if v2 == 'A':
    dogresult[10] = 1
elif v1 == 'J':
    dogresult[11] = 1
if v3 == 'I':
    dogresult[12] = 1
elif v3 == 'O':
    dogresult[14] = 1
elif v3 == 'IO':
    dogresult[13] = 1
    
dogresult[2]=v5
dogresult[3]=v6
dogresult[4]=v7

if v8 == '10u':
    dogresult[15] = 1
elif v8 == '30o':
    dogresult[16] = 1
elif v8 == '30u':
    dogresult[17] = 1
    
dogresult[5] = v10
dogresult[6] = v11
#정답컬럼 삭제
dogresult.drop('이름', inplace= True)

# doglist = X

pred_1= XGB.predict(dogresult)
socore_1 = (metrics.accuracy_score(xgb_pred,dogresult)*100)




#     dog.iloc[0,:] = pd.DataFrame({'크기':[v1],'주거환경':[v2],'실내실외':[v3], '외로움':[v4],'운동량':[v5],
#                     '털빠짐':[v6], '친화성(애교)':[v7],'의료비용':[v8],'개짖음':[v10],'집지키기':[v11]
#                    })
 
#     onehot_column = ['크기','거주환경','실내실외','의료비용']
#     for col in onehot_column:
#         dummy = pd.get_dummies(dog[col],prefix = col)
#         # 원핫인코딩된 결과물을 기존의 자료와 합치기
#         dog = pd.concat([dog,dummy], axis = 1)
#         # 글자형 컬럼(원핫인코딩 진행한) 삭제
#         dog.drop(col,axis = 1, inplace = True)
    
    
    
    
#     1.반려견 크기
#     2 주거환경
#     3 실내 실외
#     4 외로움
#     5 활동적이고 노는걸 좋아하냐
#     6 털빠지냐
#     7 사랑을 적극적으로 표현하냐
#     8 의료비용 지출
#     10 짖는 빈도
#     11집지키기 능력
    

    
    url = 'http://localhost:8090/rebuild/testresult.jsp?v1='+v1+'&v2='+v2+'&v3='+v3+'&v4='+v4+'&v5='+v5+'&v6='+v6+'&v7='+v7+'&v8='+v8+'&v10='+v10+'&v11='+v11
    return redirect(url)


if __name__ == '__main__':
    app.run(host='127.0.0.1', port='5000') 

In [ ]:
   #예시
v1 = 'S'
v2 = 'A'
v3 = 'I'
v4 = 4
v5 = 4
v6 = 3
v7 =2
v8 = '10u'
v10 = 2
v11 = 5


In [ ]:
dogresult

In [ ]:
dogresult = dogresult.iloc[0,:]

#정답지 백지화
for i in range(len(dogresult)):
    dogresult[i] = 0
#정답 입력
if v1 == 'S':
    dogresult[9] =1
elif v1 == 'M':
    dogresult[8] =1
    

elif v1 == 'L':
    dogresult[7]= 1
if v2 == 'A':
    dogresult[10] = 1
elif v1 == 'J':
    dogresult[11] = 1
if v3 == 'I':
    dogresult[12] = 1
elif v3 == 'O':
    dogresult[14] = 1
elif v3 == 'IO':
    dogresult[13] = 1
    
dogresult[2]=v5
dogresult[3]=v6
dogresult[4]=v7

if v8 == '10u':
    dogresult[15] = 1
elif v8 == '30o':
    dogresult[16] = 1
elif v8 == '30u':
    dogresult[17] = 1
    
dogresult[5] = v10
dogresult[6] = v11
#정답컬럼 삭제
dogresult.drop('이름', inplace= True)

# # dogresultlist = X
# dogresult1 = np.array(dogresult)
# dogresult1 = dogresult1.reshape(1,-1)
# knn_dogresult = knn_model.predict(dogresult)
# xgb_dogresult = XGB.predict(dogresult)
# lgb_dogresult = LGB.predict(dogresult)
# new_dogresult = np.array([knn_dogresult,xgb_dogresult,lgb_dogresult])
# new_dogresult.shape
# new_dogresult = np.trdogresultpose(new_dogresult)
# new_dogresult.shape

# pred_1= XGB.predict(dogresultresult)
# socore_1 = (metrics.accuracy_score(xgb_pred,dogresult)*100)